<a href="https://colab.research.google.com/github/meghmodi2810/Machine-Learning-Projects/blob/main/WeatherForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [164]:
import requests
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from datetime import datetime, timedelta

import pytz

In [165]:
API_KEY = 'a6632f7c2c6f9fafb60831c544cf7dd4'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

In [166]:
# prompt: get_current_weather function

def get_current_weather(city):
  """
  Fetches the current weather data for a given city using the OpenWeatherMap API.

  Args:
    city: The name of the city.

  Returns:
    A dictionary containing the current weather data, or None if an error occurs.
  """
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url)
  data = response.json()

  if response.status_code != 200:
    print("Failed to get data:", data)
    return None

  return {
      'city': data['name'],
      'current_temp' : round(data['main']['temp']),
      'feels_like' : round(data['main']['feels_like']),
      'temp_min' : round(data['main']['temp_min']),
      'temp_max' : round(data['main']['temp_max']),
      'humidity' : round(data['main']['humidity']),
      'description' : data['weather'][0]['description'],
      'country' : data['sys']['country'],
      'wind_gust_dir' : data['wind']['deg'],
      'pressure' : data['main']['pressure'],
      'Wind_Gust_Speed' : data['wind']['speed'],
      'sunrise' : datetime.fromtimestamp(data['sys']['sunrise']),
      'sunset' : datetime.fromtimestamp(data['sys']['sunset']),
      'wind_speed' : round(data['wind']['speed'])
  }

In [167]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

In [168]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]

  y = data['RainTomorrow']

  return X, y, le

In [169]:
def train_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train)

  X_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, X_pred)
  print(f'Mean Squared Error: {mse}')

  return model

In [170]:
def prepare_regression_data(data, feature):
  ''' X to store the feature values, Y will store target values to predict'''
  X, y = [], []

  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  return np.array(X).reshape(-1, 1), np.array(y)



In [171]:
def train_regression_model(X, y):
  model = RandomForestRegressor(random_state=42, n_estimators=100)
  model.fit(X, y)

  return model

In [172]:
def predict_future(model, curr):
  predictions = [curr]

  for i in range(5):
    next = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next[0])

  return predictions[1:]

In [173]:
def weather_view():
  city = input("Enter the city name: ")
  current_weather = get_current_weather(city)

  if current_weather is None:
    print("Error: Could not retrieve weather data.")
    return

  historical_data = read_historical_data('sample_data/weather.csv')
  # print(historical_data.columns)

  X, y, le = prepare_data(historical_data)
  model = train_model(X, y)
  wind_deg = current_weather['wind_gust_dir'] % 360

  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
      ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75)
  ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['wind_speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp']
  }

  df = pd.DataFrame([current_data])
  rain_predict = model.predict(df)[0]


  X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  temp_model = train_regression_model(X_temp, y_temp)

  X_humidity, y_humidity = prepare_regression_data(historical_data, 'Humidity')
  humidity_model = train_regression_model(X_humidity, y_humidity)

  future_temp = predict_future(temp_model, current_weather['current_temp'])
  future_humidity = predict_future(humidity_model, current_weather['humidity'])

  timezone = pytz.timezone('Asia/Kolkata')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  print(f"City: {current_weather['city']}")
  print(f"Country: {current_weather['country']}")
  print(f"Current Temprature: {current_weather['current_temp']} °C")
  print(f"Minimum Temprature: {current_weather['temp_min']} °C")
  print(f"Maximum Temprature: {current_weather['temp_max']} °C")
  print(f"Humidity: {current_weather['humidity']}")
  print(f"Pressure: {current_weather['pressure']}")
  print(f"Feels Like: {current_weather['feels_like']}")
  print(f"Description: {current_weather['description']}")
  print(f"Wind Speed: {current_weather['Wind_Gust_Speed']}")
  print(f"Wind Gust Direction: {current_weather['wind_gust_dir']}")
  print(f"Rain Tomorrow: {'Yes' if rain_predict == 1 else 'No'}")
  print(f"Sunrise: {current_weather['sunrise']}")
  print(f"Sunset: {current_weather['sunset']}")


  print("\nFuture Temprature Predictions: ")
  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)} °C")

  print("\nFuture Humidity Predictions: ")
  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)} °C")

In [175]:
weather_view()

Enter the city name: Mumbai
Index(['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity',
       'Pressure', 'Temp', 'RainTomorrow'],
      dtype='object')
Mean Squared Error: 0.1506849315068493
City: Mumbai
Country: IN
Current Temprature: 29 °C
Minimum Temprature: 29 °C
Maximum Temprature: 29 °C
Humidity: 79
Pressure: 1006
Feels Like: 34
Description: mist
Wind Speed: 4.12
Wind Gust Direction: 250
Rain Tomorrow: Yes
Sunrise: 2025-06-20 00:31:54
Sunset: 2025-06-20 13:48:22

Future Temprature Predictions: 
21:00: 22.1 °C
22:00: 23.3 °C
23:00: 19.0 °C
00:00: 24.5 °C
01:00: 24.5 °C

Future Humidity Predictions: 
21:00: 54.0 °C
22:00: 58.1 °C
23:00: 54.2 °C
00:00: 58.1 °C
01:00: 54.2 °C
